## 📦 Setup & Imports

First, we import the necessary libraries. EventFlow uses **Polars** for fast DataFrame operations and **NumPy** for array handling.

In [1]:
# Core libraries
import polars as pl
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path

# EventFlow imports
import sys
sys.path.insert(0, str(Path.cwd().parent / "src"))

from eventflow.core.adapters import (
    # Each adapter has a Config class for settings and an Adapter class for conversion
    TableAdapter, TableAdapterConfig,
    SequenceAdapter, SequenceAdapterConfig,
    RasterAdapter, RasterAdapterConfig,
    GraphAdapter, GraphAdapterConfig,
    StreamAdapter, StreamAdapterConfig,
)

print("✅ EventFlow adapters loaded successfully!")
print(f"\nAvailable adapters:")
print("  • TableAdapter    - For regression models (GLM, XGBoost)")
print("  • SequenceAdapter - For sequence models (LSTM, Transformer)")
print("  • RasterAdapter   - For grid models (CNN, ConvLSTM)")
print("  • GraphAdapter    - For graph models (GCN, GAT)")
print("  • StreamAdapter   - For continuous-time models (Neural ODE)")

✅ EventFlow adapters loaded successfully!

Available adapters:
  • TableAdapter    - For regression models (GLM, XGBoost)
  • SequenceAdapter - For sequence models (LSTM, Transformer)
  • RasterAdapter   - For grid models (CNN, ConvLSTM)
  • GraphAdapter    - For graph models (GCN, GAT)
  • StreamAdapter   - For continuous-time models (Neural ODE)


---
## 📊 Step 1: Generate Sample Crime Data

We'll create synthetic Chicago crime data that mimics the real dataset structure. Each record represents a single crime incident with:

- **Spatial info**: Latitude, longitude, beat, district
- **Temporal info**: Timestamp of when the crime occurred
- **Categorical info**: Crime type (theft, battery, etc.)
- **Binary flags**: Whether an arrest was made, domestic incident

### Why Synthetic Data?
Using synthetic data lets us:
1. Run the demo without downloading external files
2. Control the data characteristics for clear demonstrations
3. Ensure reproducibility with `np.random.seed(42)`

In [2]:
# Set seed for reproducibility
np.random.seed(42)
n_records = 1000

# Chicago geographic bounds (approximate)
# These define the spatial extent of our synthetic data
lat_min, lat_max = 41.64, 42.02  # ~42km north-south
lon_min, lon_max = -87.94, -87.52  # ~32km east-west

# Common crime types in Chicago
crime_types = ["THEFT", "BATTERY", "ASSAULT", "BURGLARY", "ROBBERY", "NARCOTICS"]

# Generate timestamps spread over 30 days
base_date = datetime(2024, 1, 1)
dates = [base_date + timedelta(hours=np.random.randint(0, 24*30)) for _ in range(n_records)]

# Create the DataFrame
crime_df = pl.DataFrame({
    "case_id": [f"JE{100000 + i}" for i in range(n_records)],
    "timestamp": dates,
    "latitude": np.random.uniform(lat_min, lat_max, n_records),
    "longitude": np.random.uniform(lon_min, lon_max, n_records),
    "primary_type": np.random.choice(crime_types, n_records),
    "beat": np.random.randint(100, 2500, n_records),
    "district": np.random.randint(1, 26, n_records),
    "domestic": np.random.choice([True, False], n_records, p=[0.15, 0.85]),
    "arrest": np.random.choice([True, False], n_records, p=[0.25, 0.75]),
})

print(f"📋 Generated {len(crime_df):,} crime records")
print(f"📅 Date range: {crime_df['timestamp'].min()} to {crime_df['timestamp'].max()}")
print(f"🗺️  Spatial bounds: ({lat_min:.2f}, {lon_min:.2f}) to ({lat_max:.2f}, {lon_max:.2f})")
print(f"\n🔍 Sample records:")
crime_df.head(5)

📋 Generated 1,000 crime records
📅 Date range: 2024-01-01 00:00:00 to 2024-01-30 23:00:00
🗺️  Spatial bounds: (41.64, -87.94) to (42.02, -87.52)

🔍 Sample records:


case_id,timestamp,latitude,longitude,primary_type,beat,district,domestic,arrest
str,datetime[μs],f64,f64,str,i32,i32,bool,bool
"""JE100000""",2024-01-05 06:00:00,41.992051,-87.828537,"""ROBBERY""",2107,17,false,false
"""JE100001""",2024-01-19 03:00:00,41.914207,-87.865338,"""NARCOTICS""",1463,10,true,true
"""JE100002""",2024-01-12 06:00:00,41.658276,-87.64952,"""ASSAULT""",2487,1,false,true
"""JE100003""",2024-01-05 10:00:00,41.936976,-87.680418,"""BURGLARY""",303,3,false,true
"""JE100004""",2024-01-03 23:00:00,41.954618,-87.67406,"""ASSAULT""",260,8,false,false


---
## 🗺️ Step 2: Spatial Binning & Temporal Aggregation

Most ML models for crime prediction work with **aggregated counts** rather than individual incidents. We need to:

1. **Divide space into grid cells** - Create a 10×10 grid over Chicago
2. **Group by time windows** - Aggregate to daily counts
3. **Compute summary statistics** - Total events, arrests, etc.

### The Grid System
```
┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ 0 │ 1 │ 2 │ 3 │ 4 │ 5 │ 6 │ 7 │ 8 │ 9 │  ← lon_bin
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│10 │11 │12 │...│   │   │   │   │   │19 │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│...│   │   │   │ cell_id = lat_bin*10 + lon_bin
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│90 │91 │92 │...│   │   │   │   │   │99 │
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘
  ↑
lat_bin
```

In [3]:
# Define grid dimensions
n_lat_bins, n_lon_bins = 10, 10
print(f"🗺️  Creating a {n_lat_bins}×{n_lon_bins} spatial grid ({n_lat_bins * n_lon_bins} cells total)")

# Add spatial bin columns and temporal features
crime_df = crime_df.with_columns([
    # Compute which grid cell each crime falls into
    ((pl.col("latitude") - lat_min) / (lat_max - lat_min) * n_lat_bins).floor().cast(pl.Int32).alias("lat_bin"),
    ((pl.col("longitude") - lon_min) / (lon_max - lon_min) * n_lon_bins).floor().cast(pl.Int32).alias("lon_bin"),
    # Extract date (for daily aggregation)
    pl.col("timestamp").dt.date().alias("date"),
    # Extract hour (useful for temporal patterns)
    pl.col("timestamp").dt.hour().alias("hour"),
])

# Create unique cell ID from lat/lon bins
crime_df = crime_df.with_columns(
    (pl.col("lat_bin") * n_lon_bins + pl.col("lon_bin")).alias("cell_id")
)

# Aggregate to daily counts per cell
daily_counts = crime_df.group_by(["cell_id", "date", "lat_bin", "lon_bin"]).agg([
    pl.len().alias("event_count"),           # Total crimes in this cell-day
    pl.col("arrest").sum().alias("arrest_count"),     # How many led to arrests
    pl.col("domestic").sum().alias("domestic_count"), # Domestic incidents
    pl.col("latitude").mean().alias("centroid_lat"),  # Cell centroid
    pl.col("longitude").mean().alias("centroid_lon"),
]).sort(["date", "cell_id"])

print(f"\n📊 Aggregation Results:")
print(f"   • Original records: {len(crime_df):,}")
print(f"   • Aggregated rows: {len(daily_counts):,} (cell × day combinations)")
print(f"   • Unique cells with events: {daily_counts['cell_id'].n_unique()}")
print(f"   • Date range: {daily_counts['date'].min()} to {daily_counts['date'].max()}")
print(f"\n🔍 Sample aggregated data:")
daily_counts.head(5)

🗺️  Creating a 10×10 spatial grid (100 cells total)

📊 Aggregation Results:
   • Original records: 1,000
   • Aggregated rows: 858 (cell × day combinations)
   • Unique cells with events: 100
   • Date range: 2024-01-01 to 2024-01-30

🔍 Sample aggregated data:


cell_id,date,lat_bin,lon_bin,event_count,arrest_count,domestic_count,centroid_lat,centroid_lon
i32,date,i32,i32,u32,u32,u32,f64,f64
8,2024-01-01,0,8,1,1,0,41.673646,-87.577965
10,2024-01-01,1,0,1,0,0,41.692895,-87.926803
20,2024-01-01,2,0,1,0,0,41.735961,-87.937182
21,2024-01-01,2,1,1,0,0,41.722849,-87.896262
24,2024-01-01,2,4,1,0,0,41.72837,-87.748934


---
## 📈 Adapter 1: TableAdapter (For Regression Models)

### When to Use
Use `TableAdapter` when you want to predict **counts** or **rates** using:
- **Poisson/Negative Binomial GLM** - Classic count regression
- **XGBoost/LightGBM** - Gradient boosting
- **Random Forest** - Ensemble methods
- **Linear models** - Ridge, Lasso

### Key Features
- **Offset column** - For exposure adjustment (e.g., population at risk)
- **Weight column** - For observation weighting
- **Intercept** - Optional intercept column for GLMs
- **get_X_y()** - Returns numpy arrays ready for sklearn

### Configuration Options
```python
TableAdapterConfig(
    target_col="event_count",     # Column to predict
    feature_cols=[...],            # Features to include (auto-detected if None)
    offset_col="exposure",         # Log-offset for Poisson GLM
    weight_col="weight",           # Observation weights
    include_intercept=True,        # Add column of 1s
    categorical_encoding="onehot", # How to handle categoricals
)
```

In [4]:
# Prepare data for TableAdapter
# Add day-of-week feature and uniform exposure
table_df = daily_counts.with_columns([
    pl.col("date").dt.weekday().alias("day_of_week"),  # 0=Monday, 6=Sunday
    pl.lit(1.0).alias("exposure"),  # Uniform exposure (could be population)
])

# Configure the adapter
table_config = TableAdapterConfig(
    target_col="event_count",  # What we want to predict
    feature_cols=["lat_bin", "lon_bin", "day_of_week", "arrest_count", "domestic_count"],
    offset_col="exposure",     # For Poisson regression: log(exposure) offset
    include_intercept=True,    # Add intercept column for GLM
)

# Convert!
table_adapter = TableAdapter(table_config)
table_output = table_adapter.convert(table_df)

print("📊 TableAdapter Output:")
print(f"   • Data shape: {table_output.data.shape}")
print(f"   • Feature names: {table_output.feature_names}")
print(f"   • Target column: '{table_output.target}'")
print(f"   • Offset column: '{table_output.offset}'")

# Get sklearn-ready arrays
X, y = table_output.get_X_y()
print(f"\n🔢 NumPy Arrays (ready for sklearn):")
print(f"   • X shape: {X.shape} (samples × features)")
print(f"   • y shape: {y.shape} (samples,)")
print(f"   • X dtype: {X.dtype}")
print(f"\n📋 First 3 rows of X:")
print(X[:3])

2025-12-01 03:09:46,284 - eventflow.core.adapters.table - INFO - Converting EventFrame to table format
2025-12-01 03:09:46,286 - eventflow.core.adapters.table - INFO - Created table with 858 rows, 6 features


📊 TableAdapter Output:
   • Data shape: (858, 8)
   • Feature names: ['_intercept', 'lat_bin', 'lon_bin', 'day_of_week', 'arrest_count', 'domestic_count']
   • Target column: 'event_count'
   • Offset column: 'exposure'

🔢 NumPy Arrays (ready for sklearn):
   • X shape: (858, 6) (samples × features)
   • y shape: (858,) (samples,)
   • X dtype: float32

📋 First 3 rows of X:
[[1. 0. 8. 1. 1. 0.]
 [1. 1. 0. 1. 0. 0.]
 [1. 2. 0. 1. 0. 0.]]


### Example: Fitting a Poisson GLM

```python
from sklearn.linear_model import PoissonRegressor

X, y = table_output.get_X_y()
model = PoissonRegressor()
model.fit(X, y)
predictions = model.predict(X)
```

---

## 🔄 Adapter 2: SequenceAdapter (For RNN/Transformer)

### When to Use
Use `SequenceAdapter` when you want to model **temporal patterns** at each location:
- **LSTM/GRU** - Recurrent neural networks
- **Transformer** - Self-attention models
- **Temporal Convolutional Networks** - 1D convolutions over time

### Key Features
- **Padding** - Handles variable-length sequences (left or right padding)
- **Attention masks** - Indicates which positions are real vs. padded
- **Time encoding** - Optional positional or sinusoidal embeddings

### Output Shape
```
sequences: (n_locations, max_time_steps, n_features)
masks:     (n_locations, max_time_steps)  # 1=real, 0=padded
lengths:   (n_locations,)  # actual sequence length per location
```

In [5]:
# Prepare sequence data
sequence_data = daily_counts.select(["cell_id", "date", "event_count"]).sort(["cell_id", "date"])

print(f"📊 Input: {len(sequence_data)} rows covering {sequence_data['cell_id'].n_unique()} locations")

# Configure the adapter
seq_config = SequenceAdapterConfig(
    spatial_col="cell_id",       # Group sequences by this column
    timestamp_col="date",         # Order within each group
    feature_cols=["event_count"], # Features at each time step
    sequence_length=30,           # Pad/truncate to this length
    padding_value=0.0,            # Value for padded positions
    padding_side="right",         # Pad at the end (common for RNNs)
    return_masks=True,            # Return attention masks
)

# Convert!
seq_adapter = SequenceAdapter(seq_config)
seq_output = seq_adapter.convert(sequence_data)

print(f"\n🔄 SequenceAdapter Output:")
print(f"   • Sequences shape: {seq_output.sequences.shape}")
print(f"     → {seq_output.sequences.shape[0]} locations × {seq_output.sequences.shape[1]} time steps × {seq_output.sequences.shape[2]} features")
print(f"   • Masks shape: {seq_output.masks.shape}")
print(f"   • Sequence lengths (first 5): {seq_output.lengths[:5]}")

# Show an example sequence
print(f"\n📋 Example: First location's sequence (first 10 time steps):")
print(f"   Values: {seq_output.sequences[0, :10, 0]}")
print(f"   Mask:   {seq_output.masks[0, :10]}  (1=real data, 0=padding)")

2025-12-01 03:09:53,173 - eventflow.core.adapters.sequence - INFO - Converting EventFrame to sequence format
2025-12-01 03:09:53,198 - eventflow.core.adapters.sequence - INFO - Created 100 sequences with max_length=30, n_features=1


📊 Input: 858 rows covering 100 locations

🔄 SequenceAdapter Output:
   • Sequences shape: (100, 30, 1)
     → 100 locations × 30 time steps × 1 features
   • Masks shape: (100, 30)
   • Sequence lengths (first 5): [10  9  8  9 10]

📋 Example: First location's sequence (first 10 time steps):
   Values: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
   Mask:   [ True  True  True  True  True  True  True  True  True  True]  (1=real data, 0=padding)


### Example: Using with PyTorch LSTM

```python
import torch
import torch.nn as nn

# Convert to tensors
sequences = torch.from_numpy(seq_output.sequences)  # (batch, seq_len, features)
masks = torch.from_numpy(seq_output.masks)          # (batch, seq_len)

# Create LSTM
lstm = nn.LSTM(input_size=1, hidden_size=32, batch_first=True)
output, (h_n, c_n) = lstm(sequences)
```

---

## 🖼️ Adapter 3: RasterAdapter (For CNN Models)

### When to Use
Use `RasterAdapter` when you want to treat the city as an **image** and exploit spatial patterns:
- **CNN** - 2D convolutions for spatial patterns
- **ConvLSTM** - Spatio-temporal convolutions
- **U-Net** - Encoder-decoder for dense prediction

### Key Features
- **Channel-first/last** - PyTorch (C,H,W) vs TensorFlow (H,W,C)
- **Multiple channels** - Stack different features as channels
- **Fill value** - Handle cells with no data
- **Normalization** - Optional channel normalization

### Output Shape
```
Channel-first (PyTorch): (n_timesteps, n_channels, height, width)
Channel-last (TensorFlow): (n_timesteps, height, width, n_channels)
```

In [6]:
# Configure the adapter
raster_config = RasterAdapterConfig(
    grid_col="cell_id",          # Maps to grid position
    timestamp_col="date",         # Creates time dimension
    feature_cols=["event_count", "arrest_count"],  # Each becomes a channel
    grid_shape=(n_lat_bins, n_lon_bins),  # 10×10 grid
    channel_first=True,           # PyTorch format (C, H, W)
    fill_value=0.0,               # Empty cells get 0
    normalize=False,              # Keep raw counts
)

# Convert!
raster_adapter = RasterAdapter(raster_config)
raster_output = raster_adapter.convert(daily_counts)

print(f"🖼️ RasterAdapter Output:")
print(f"   • Raster shape: {raster_output.raster.shape}")
print(f"     → {raster_output.raster.shape[0]} time steps")
print(f"     → {raster_output.raster.shape[1]} channels: {raster_output.channel_names}")
print(f"     → {raster_output.raster.shape[2]}×{raster_output.raster.shape[3]} spatial grid")
print(f"   • Timestamps: {len(raster_output.timestamps)} dates")

# Visualize one time slice
print(f"\n📋 First day, 'event_count' channel (10×10 grid):")
print(raster_output.raster[0, 0])  # First timestep, first channel

2025-12-01 03:10:01,557 - eventflow.core.adapters.raster - INFO - Converting EventFrame to raster format
2025-12-01 03:10:01,564 - eventflow.core.adapters.raster - INFO - Created raster with shape (30, 2, 10, 10): 30 timesteps, 2 channels, 10x10 grid


🖼️ RasterAdapter Output:
   • Raster shape: (30, 2, 10, 10)
     → 30 time steps
     → 2 channels: ['event_count', 'arrest_count']
     → 10×10 spatial grid
   • Timestamps: 30 dates

📋 First day, 'event_count' channel (10×10 grid):
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [2. 1. 0. 1. 1. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 2. 0. 1. 0. 0.]
 [0. 2. 2. 1. 0. 0. 0. 1. 0. 0.]]


### Example: Using with PyTorch CNN

```python
import torch
import torch.nn as nn

# Shape: (batch, channels, height, width)
x = torch.from_numpy(raster_output.raster)  # (30, 2, 10, 10)

# Simple CNN for crime prediction
model = nn.Sequential(
    nn.Conv2d(2, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 1, kernel_size=3, padding=1),  # Predict next day
)
predictions = model(x)
```

---

## 🕸️ Adapter 4: GraphAdapter (For GNN Models)

### When to Use
Use `GraphAdapter` when you want to model **relationships between locations**:
- **GCN** - Graph Convolutional Networks
- **GAT** - Graph Attention Networks
- **GraphSAGE** - Inductive learning on graphs

### Key Features
- **Adjacency types** - Spatial proximity, temporal, or both
- **Edge features** - Optional features on edges
- **Self-loops** - Include self-connections
- **Normalization** - Symmetric or row normalization

### Output Components
```python
node_features: (n_nodes, n_features)  # Feature vector per node
edge_index: (2, n_edges)              # COO format edges [src, dst]
adjacency: (n_nodes, n_nodes)         # Dense adjacency matrix
```

In [7]:
# Aggregate to get one feature vector per cell (node)
node_features_df = daily_counts.group_by("cell_id").agg([
    pl.col("event_count").sum().alias("total_events"),
    pl.col("event_count").mean().alias("avg_events"),
    pl.col("arrest_count").sum().alias("total_arrests"),
    pl.col("centroid_lat").mean().alias("lat"),
    pl.col("centroid_lon").mean().alias("lon"),
]).sort("cell_id")

print(f"📊 Node data: {len(node_features_df)} cells (nodes)")

# Configure the adapter
graph_config = GraphAdapterConfig(
    node_col="cell_id",           # Node identifier
    feature_cols=["total_events", "avg_events", "total_arrests", "lat", "lon"],
    adjacency_type="spatial",     # Connect nearby nodes
    spatial_threshold=0.05,        # ~5km radius for edges
    include_self_loops=True,       # Nodes connect to themselves
    normalize_adjacency=True,      # Normalize for GCN
)

# Convert!
graph_adapter = GraphAdapter(graph_config)
graph_output = graph_adapter.convert(node_features_df)

print(f"\n🕸️ GraphAdapter Output:")
print(f"   • Node features: {graph_output.node_features.shape}")
print(f"     → {graph_output.node_features.shape[0]} nodes × {graph_output.node_features.shape[1]} features")
print(f"   • Edge index: {graph_output.edge_index.shape}")
print(f"     → {graph_output.edge_index.shape[1]} edges total")
print(f"   • Adjacency matrix: {graph_output.adjacency.shape}")

# Show sample edges
print(f"\n📋 First 5 edges (node → node):")
for i in range(min(5, graph_output.edge_index.shape[1])):
    src, dst = graph_output.edge_index[0, i], graph_output.edge_index[1, i]
    print(f"   {src} → {dst}")

2025-12-01 03:10:13,943 - eventflow.core.adapters.graph - INFO - Converting EventFrame to graph format
2025-12-01 03:10:13,962 - eventflow.core.adapters.graph - INFO - Created graph with 100 nodes, 476 edges, 5 features


📊 Node data: 100 cells (nodes)

🕸️ GraphAdapter Output:
   • Node features: (100, 5)
     → 100 nodes × 5 features
   • Edge index: (2, 476)
     → 476 edges total
   • Adjacency matrix: (100, 100)

📋 First 5 edges (node → node):
   0 → 1
   1 → 0
   0 → 10
   10 → 0
   1 → 2


### Example: Using with PyTorch Geometric

```python
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Create PyG Data object
data = Data(
    x=torch.from_numpy(graph_output.node_features),
    edge_index=torch.from_numpy(graph_output.edge_index),
)

# Simple GCN layer
conv = GCNConv(in_channels=5, out_channels=16)
node_embeddings = conv(data.x, data.edge_index)
```

---

## ⏱️ Adapter 5: StreamAdapter (For Neural ODE / Point Processes)

### When to Use
Use `StreamAdapter` when you want to model **continuous-time dynamics**:
- **Neural ODE** - Continuous-depth networks
- **Temporal Point Processes** - Hawkes, self-exciting models
- **Continuous-time RNNs** - ODE-RNN, GRU-ODE

### Key Features
- **Inter-event times** - Time between consecutive events
- **Event types (marks)** - Categorical event labels
- **Time scaling** - Normalize, log-transform, or raw
- **Relative timestamps** - Origin at first event or custom

### Output Components
```python
timestamps: (n_events,)    # Scaled event times
inter_times: (n_events,)   # Time since previous event
states: (n_events, n_features)  # State at each event
event_types: (n_events,)   # Encoded event type
```

In [8]:
# Use individual events (not aggregated) for continuous-time modeling
stream_df = crime_df.select([
    "case_id", "timestamp", "primary_type", "latitude", "longitude"
]).sort("timestamp")

print(f"📊 Stream data: {len(stream_df)} individual events")

# Configure the adapter
stream_config = StreamAdapterConfig(
    timestamp_col="timestamp",     # Event times
    event_type_col="primary_type", # Event marks (crime type)
    state_cols=["latitude", "longitude"],  # State vector
    time_scale="normalize",        # Normalize to mean=0, std=1
    time_origin="first",           # Start from first event
    return_inter_times=True,       # Compute inter-event times
)

# Convert!
stream_adapter = StreamAdapter(stream_config)
stream_output = stream_adapter.convert(stream_df)

print(f"\n⏱️ StreamAdapter Output:")
print(f"   • Timestamps: {stream_output.timestamps.shape} (normalized)")
print(f"   • States: {stream_output.states.shape} (lat, lon per event)")
print(f"   • Inter-event times: {stream_output.inter_times.shape}")
print(f"   • Event types: {stream_output.event_types.shape}")
print(f"   • Unique event types: {np.unique(stream_output.event_types)}")

# Show sample
print(f"\n📋 First 5 events:")
print(f"   Timestamps (normalized): {stream_output.timestamps[:5]}")
print(f"   Inter-event times: {stream_output.inter_times[:5]}")
print(f"   Event types: {stream_output.event_types[:5]}")

2025-12-01 03:10:34,400 - eventflow.core.adapters.stream - INFO - Converting EventFrame to stream format
2025-12-01 03:10:34,403 - eventflow.core.adapters.stream - INFO - Created stream with 1000 events, 2 state dims


📊 Stream data: 1000 individual events

⏱️ StreamAdapter Output:
   • Timestamps: (1000,) (normalized)
   • States: (1000, 2) (lat, lon per event)
   • Inter-event times: (1000,)
   • Event types: (1000,)
   • Unique event types: [0 1 2 3 4 5]

📋 First 5 events:
   Timestamps (normalized): [-1.6646022 -1.6646022 -1.6646022 -1.6599252 -1.6599252]
   Inter-event times: [0.         0.         0.         0.00467694 0.        ]
   Event types: [3 1 2 4 1]


### Example: Using for Temporal Point Process

```python
# Inter-event times are key for TPP models
inter_times = stream_output.inter_times  # Time gaps
event_types = stream_output.event_types  # Crime types as integers

# Hawkes process likelihood uses these directly
# Neural TPP models (like THP, SAHP) use them as input sequences
```

---

## 📊 Summary: Choosing the Right Adapter

| If you want to... | Use this adapter | Output format |
|-------------------|------------------|---------------|
| Predict counts with features | `TableAdapter` | (samples, features) + (samples,) |
| Model temporal patterns per location | `SequenceAdapter` | (locations, time, features) |
| Treat the city as an image | `RasterAdapter` | (time, channels, H, W) |
| Model spatial relationships | `GraphAdapter` | node_features + edge_index |
| Model continuous-time events | `StreamAdapter` | timestamps + inter_times |

### Serialization
All adapters support saving to disk in multiple formats:
```python
# Save as NumPy
adapter.serialize(output, "model_input.npz", format="numpy")

# Save as PyTorch
adapter.serialize(output, "model_input.pt", format="pytorch")

# Save as Parquet (for tabular data)
table_adapter.serialize(table_output, "data.parquet", format="parquet")
```

---

## 🚀 Next Steps

1. **Try with real data** - Download actual Chicago crime data from the City Data Portal
2. **Add more features** - Weather, holidays, demographics
3. **Build models** - Use the outputs with your favorite ML framework
4. **Tune adapters** - Adjust grid resolution, time windows, graph connectivity

For more information, see the EventFlow documentation and API reference.